## PAD MASK IMAGE

## good method

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random

def process_and_draw_contours(image_path, original_image_path):
    def pad_with_white_line(binary_mask, pad_size=10):
        # Create a white border around the binary mask
        white_border = cv2.copyMakeBorder(binary_mask, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_CONSTANT, value=255)
        return white_border

    def preprocess_image(binary_mask):
        # Detect edges using the Canny edge detector
        edges = cv2.Canny(binary_mask, 50, 150, apertureSize=3)

        # Dilate the edges to connect nearby lines with a smaller kernel
        kernel = np.ones((3, 3), np.uint8)
        dilated_edges = cv2.dilate(edges, kernel, iterations=1)

        # Detect lines using the probabilistic Hough Line Transform
        lines = cv2.HoughLinesP(dilated_edges, 1, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=10)

        # Create a blank image to draw lines
        line_image = np.zeros_like(binary_mask)

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), 255, 1)  # Reduce thickness of lines

        # Combine the original binary mask with the lines
        completed_mask = cv2.bitwise_or(binary_mask, line_image)

        # Use morphological closing to fill any small gaps with a smaller kernel
        kernel = np.ones((3, 3), np.uint8)
        closed_mask = cv2.morphologyEx(completed_mask, cv2.MORPH_CLOSE, kernel)
        
        return closed_mask

    def is_valid_shape(contour, binary_mask):
        # Approximate the contour
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Calculate the aspect ratio of the bounding rectangle
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h if h != 0 else 0

        # Calculate the solidity of the contour
        area = cv2.contourArea(contour)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area if hull_area != 0 else 0

        # Exclude long or short thin lines
        if aspect_ratio < 0.3 or aspect_ratio > 3:
            return False

        # Criteria for valid shapes
        if 0.5 < solidity:
            return True
        return False

    def find_closed_areas_with_white_boundary(binary_mask):
        # Find contours in the binary mask
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # List to store closed areas
        closed_areas = []

        # Minimum area threshold for consideration (0.2% of the image area)
        min_area_threshold = binary_mask.shape[0] * binary_mask.shape[1] // 700

        # Maximum area threshold for exclusion (50% of the image area)
        max_area_threshold = binary_mask.shape[0] * binary_mask.shape[1] // 2

        # Image dimensions
        height, width = binary_mask.shape

        # Loop over the contours and filter out closed areas
        for i, contour in enumerate(contours):
            # Check if the contour is closed and surrounded by a white boundary or connected to the edge
            if (hierarchy[0][i][2] == -1 and hierarchy[0][i][3] != -1) or cv2.contourArea(contour) > min_area_threshold:
                # Further filter to exclude small closed areas that are just a line
                if min_area_threshold < cv2.contourArea(contour) < max_area_threshold:
                    # Check if the contour is a valid shape
                    if is_valid_shape(contour, binary_mask):
                        # Get bounding rectangle of the contour
                        x, y, w, h = cv2.boundingRect(contour)
                        # Exclude white regions
                        mask_roi = binary_mask[y:y+h, x:x+w]
                        if not np.all(mask_roi == 255):
                            # print(contour)
                            # print("end of contour\n\n\n")
                            closed_areas.append(contour)

        return closed_areas

    def draw_contours(image, contours):
        # Create a blank image to draw contours
        contour_image = np.zeros_like(image)
        contour_image = cv2.cvtColor(contour_image, cv2.COLOR_GRAY2BGR)  # Convert to color image

        # Draw and fill contours with random colors, avoiding black
        for contour in contours:
            while True:
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                # Ensure the color is not black or very dark
                if sum(color) > 100:
                    break
            cv2.drawContours(contour_image, [contour], -1, color, cv2.FILLED)

        return contour_image

    # Load the binary mask image
    binary_mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image is loaded correctly
    if binary_mask is None:
        print("Error: Could not load image.")
        return

    # Pad the binary mask with a white line around it
    padded_mask = pad_with_white_line(binary_mask, pad_size=7)

    # Load the original image
    original_image = cv2.imread(original_image_path)

    # Check if the original image is loaded correctly
    if original_image is None:
        print("Error: Could not load original image.")
        return

    # Preprocess the binary mask to complete lines
    processed_mask = preprocess_image(padded_mask)

    # Find closed areas with white boundary
    closed_areas = find_closed_areas_with_white_boundary(processed_mask)

    # Draw closed areas
    contour_image = draw_contours(padded_mask, closed_areas)

    # Display the original image, original binary mask, the processed mask, and the contour image
    plt.figure(figsize=(20, 6))
    plt.subplot(1, 4, 1)
    plt.title('Original Image')
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))

    plt.subplot(1, 4, 2)
    plt.title('Predicted Mask')
    plt.imshow(binary_mask, cmap='gray')

    plt.subplot(1, 4, 3)
    plt.title('Processed Mask')
    plt.imshow(processed_mask, cmap='gray')

    plt.subplot(1, 4, 4)
    plt.title('Closed Areas as polygon')
    plt.imshow(contour_image)

    plt.show()

# Get the list of mask images
mask_path = './saved_merged_ir_goodmedium_ed_64_2_images/'
# mask_path = './saved_merged_goodir_images/'
original_path = './original_dataset/'
mask_images = os.listdir(mask_path)

# Get all jpg images from nested folders in the original dataset
original_images = []
for root, dirs, files in os.walk(original_path):
    for file in files:
        if file.endswith('.jpg'):
            original_images.append(os.path.join(root, file))

# Process each image
for img in mask_images:
    img_name = os.path.basename(img)[:-4]
    matching_original_image = next((orig for orig in original_images if os.path.basename(orig)[:-4] == img_name), None)
    if matching_original_image:
        process_and_draw_contours(f'{mask_path}{img}', matching_original_image)
